In [4]:
import pandas as pd

# Load the JSON file into a DataFrame
file_path = 'kohls_data.json'
df = pd.read_json(file_path)

# Display the first few rows of the DataFrame to verify
df.head()

,payload
categories,"[{'ID': '1682477778193', 'name': 'Sale + Must-..."


In [5]:
# Since the JSON seems to be deeply nested, let's try to normalize the 'categories' part of the payload
# We need to first extract the 'categories' data from the payload column
import json

# Load the JSON data from the file
with open(file_path, 'r') as file:
    data = json.load(file)

# Normalize the 'categories' part of the JSON
categories_data = pd.json_normalize(data, record_path=['payload', 'categories'])

# Display the first few rows of the categories DataFrame
categories_data.head()

,ID,name,seoURL,type,categories,imageURL.bg-white,imageURL.bg-dark
0,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,"[{'ID': '1952501926573', 'name': 'Sale + Must-...",NaN,NaN
1,1776937150310,Holiday Gift Shop,/catalog/gift-ideas-holiday.jsp?CN=Feature:Gif...,Catalog,"[{'ID': '1804910701382', 'name': 'Shop All Hol...",NaN,NaN
2,1370957749499,Home,/sale-event/for-the-home.jsp?cc=for_thehome-TN...,Category,"[{'ID': '1362509278893', 'name': 'CLEARANCE', ...",NaN,NaN
3,1258955308138,Women,/sale-event/womens-clothing.jsp?cc=wms-TN1.0-S...,Category,"[{'ID': '1092699823557', 'name': 'New @ Kohl's...",NaN,NaN
4,1974847218651,Men,/sale-event/mens-clothing.jsp?cc=mens-TN1.0-S-men,Category,"[{'ID': '1743303242851', 'name': 'Men's Clothi...",NaN,NaN


In [8]:
# The categories column contains nested lists of dictionaries, so we will explode this column
# to turn each list entry into a row.

# First, let's make sure the 'categories' column contains lists for exploding
categories_data['categories'] = categories_data['categories'].apply(lambda x: x if isinstance(x, list) else [])

# Now we can explode the 'categories' column
exploded_categories = categories_data.explode('categories')

# We'll normalize the exploded 'categories' data to turn the dictionary entries into DataFrame columns
exploded_categories_normalized = pd.json_normalize(exploded_categories['categories'])

# Combine the normalized category data with the original data
exploded_categories.reset_index(drop=True, inplace=True)
exploded_categories_normalized.reset_index(drop=True, inplace=True)
exploded_categories_combined = pd.concat([exploded_categories.drop('categories', axis=1), exploded_categories_normalized], axis=1)

exploded_categories_combined.head()


,ID,name,seoURL,type,imageURL.bg-white,imageURL.bg-dark,ID,name,seoURL,type,categories
0,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,1952501926573,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN
1,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,1710577965595,Home,/catalog/for-the-home.jsp?CN=Activity:For%20th...,Catalog,NaN
2,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,1984266275788,Womens & Juniors,/catalog/juniors.jsp?CN=Gender:Womens+Gender:J...,Catalog,NaN
3,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,1192969608649,Mens,/catalog/mens.jsp?CN=Gender:Mens+Assortment:Mu...,Catalog,NaN
4,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,1537083028105,Kids,/catalog/kids-clothing.jsp?CN=AgeAppropriate:K...,Catalog,NaN


In [11]:
# Remove duplicate columns by first identifying columns with the same name but different values
# We will keep the first occurrence of each column name and drop subsequent ones

# Dropping duplicate columns by name and keeping the first
exploded_categories_combined = exploded_categories_combined.loc[:, ~exploded_categories_combined.columns.duplicated()]

# Display the cleaned DataFrame without duplicate columns
exploded_categories_combined.head()


,ID,name,seoURL,type,imageURL.bg-white,imageURL.bg-dark,categories
0,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,NaN
1,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,NaN
2,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,NaN
3,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,NaN
4,1682477778193,Sale + Must-Have Prices,/catalog/catalog.jsp?CN=Assortment:Must%20Have...,Catalog,NaN,NaN,NaN


In [12]:
exploded_categories_combined.to_csv('kohls-data.csv')

In [16]:
def extract_categories(data, key='categories'):
    """
    Recursive function to extract categories from nested JSON data.
    """
    if isinstance(data, dict):
        for k, v in data.items():
            if k == key:
                yield v
            elif isinstance(v, (dict, list)):
                yield from extract_categories(v)
    elif isinstance(data, list):
        for item in data:
            yield from extract_categories(item)

# Extract categories from the JSON data
categories_data = list(extract_categories('kohls_data.json'))

# Display a sample of the extracted categories
categories_sample = categories_data[:5]  # Display first 5 categories for inspection
categories_sample


[]

In [15]:
# Extracting distinct product names from the categories data
product_names = set()

for category in categories_data:
    if isinstance(category, list):
        for item in category:
            product_name = item.get('name')
            if product_name:
                product_names.add(product_name)
    elif isinstance(category, dict):
        product_name = category.get('name')
        if product_name:
            product_names.add(product_name)

# Convert the set to a sorted list for better readability
sorted_product_names = sorted(list(product_names))

sorted_product_names[:10]  # Displaying first 10 product names as a sample



[]